In [1]:
# set environment variables before importing any other code (in particular the openai module)
from dotenv import load_dotenv

env_path = './.env'
load_dotenv(dotenv_path=env_path)

import os

In [9]:
from openai import AsyncAzureOpenAI
import jinja2
import pathlib

from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery #RawVectorQuery

# from streaming_utils import add_context_to_streamed_response

In [3]:
# pip show azure-search-documents 

Name: azure-search-documents
Version: 11.5.1
Summary: Microsoft Azure Cognitive Search Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents
Author: Microsoft Corporation
Author-email: ascl@microsoft.com
License: MIT License
Location: /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages
Requires: azure-common, azure-core, isodate, typing-extensions
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [14]:
async def get_documents(query, num_docs=5):
    #  retrieve documents relevant to the user's question from Cognitive Search
    search_client = SearchClient(
        endpoint=os.environ["AZURE_AI_SEARCH_ENDPOINT"],
        credential=AzureKeyCredential(os.environ["AZURE_AI_SEARCH_KEY"]),
        index_name=os.environ["AZURE_AI_SEARCH_INDEX_NAME"])

    print("get_documents declare AzureOpenAI.. AZURE_AI_SEARCH_INDEX_NAME:" + os.environ["AZURE_AI_SEARCH_INDEX_NAME"])
        
    async with AsyncAzureOpenAI(
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        api_key=os.environ["AZURE_OPENAI_KEY"],    
        api_version=os.environ["AZURE_OPENAI_API_VERSION"]
    ) as aclient:

        # generate a vector embedding of the user's question
        embedding = await aclient.embeddings.create(input=query,
                                                model=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT"])
        embedding_to_query = embedding.data[0].embedding

    context = ""

#     async with search_client:
#     with search_client:
    # use the vector embedding to do a vector search on the index
    vector_query = VectorizedQuery(vector=embedding_to_query, k_nearest_neighbors=num_docs, fields="contentVector")
    # vector_query = RawVectorQuery(vector=embedding_to_query, k=num_docs, fields="contentVector")
    results = search_client.search(
        search_text="",
        vector_queries=[vector_query],
        select=["id", "content"])

    for result in results:
        context += f"\n>>> From: {result['id']}\n{result['content']}"

    return context

In [15]:
async def chat_completion(messages: list[dict], stream: bool = False,
                          session_state: any = None, context: dict[str, any] = {}):
    # get search documents for the last user message in the conversation
    user_message = messages[-1]["content"]
    documents = await get_documents(user_message, context.get("num_retrieved_docs", 5))

    # make a copy of the context and modify it with the retrieved documents
    context = dict(context)
    context['documents'] = documents
    print("chat_completion already get documents:" + documents[:-1])

    # add retrieved documents as context to the system prompt
    system_message = system_message_template.render(context=context)
    messages.insert(0, {"role": "system", "content": system_message})

    print("chat_completion, declare AzureOpenAI..")
    async with AsyncAzureOpenAI(
        azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
        api_key=os.environ["AZURE_OPENAI_KEY"],    
        api_version=os.environ["AZURE_OPENAI_API_VERSION"]
    ) as aclient:

        # call Azure OpenAI with the system prompt and user's question
        chat_completion = await aclient.chat.completions.create(
            model=os.environ.get("AZURE_OPENAI_CHAT_DEPLOYMENT"),
            messages=messages,
            temperature=context.get("temperature", 0.7),
            stream=stream,
            max_tokens=800)

        print("chat_completion llm response:" + chat_completion.choices[0].message.content)
        response = {
            "choices": [{
                "index": 0,
                "message": {
                    "role": "assistant",
                    "content": chat_completion.choices[0].message.content
                },
            }]
        }

        # add context in the returned response
        if not stream:
            response["choices"][0]["context"] = context
        else:
            response = add_context_to_streamed_response(response, context)
    return response

In [16]:
import asyncio
from openai import AzureOpenAI
import jinja2
import pathlib
import pprint

templateLoader = jinja2.FileSystemLoader('../qa_copilot_azure_sdk/aistudio-copilot-sample/src/copilot_aisdk/')
templateEnv = jinja2.Environment(loader=templateLoader)
system_message_template = templateEnv.get_template("system-message.jinja2")

question = "which tent is the most waterproof?"

result = await chat_completion([{"role": "user", "content": question}]) #, stream=True)

print("\n\n\"")
print("===============")
pprint.pp(result)
# for r in result:
#     print(r)
#     print("\n")

get_documents declare AzureOpenAI.. AZURE_AI_SEARCH_INDEX_NAME:product-info
chat_completion already get documents:
>>> From: cHJvZHVjdF9pbmZvXzgubWQy
# Information about product item_number: 8

37) **Rating:** 4
   **Review:** I bought the Alpine Explorer Tent, and while it's waterproof and spacious, I wish it had more storage pockets. Overall, it's a good tent for camping.

38) **Rating:** 5
   **Review:** The Alpine Explorer Tent is perfect for my family's camping adventures. It's easy to set up, has great ventilation, and the gear loft is an excellent addition. Love it!

39) **Rating:** 4
   **Review:** I like the Alpine Explorer Tent, but I wish it came with a footprint. It's comfortable and has many useful features, but a footprint would make it even better. Overall, it's a great tent.

40) **Rating:** 5
   **Review:** This tent is perfect for our family camping trips. It's spacious, easy to set up, and the room divider is a great feature for added privacy. The gear loft is a nice